In [2]:
import ipywidgets as widgets
import yfinance as yf
import pandas as pd
import numpy
import math
import altair as alt
import time
import json
import datetime
import random
import defs
import time
import datetime

#missing
#fees
#dividends
#stocksplits - what happens in  a stock split(are old shares amount converted to new share amount)


## File Load

- Load File
- Get All Distinct Stokcs
- Get All Distinct Currencies

In [3]:

filewidget = widgets.Text(
    value='Finance_Positions.csv',#''
    placeholder='Transactions.csv',
    description='Transaction File Location:',
    #disabled=False
)
filewidget

Text(value='Finance_Positions.csv', description='Transaction File Location:', placeholder='Transactions.csv')

In [4]:
filename=filewidget.value

In [5]:
#filename = 'Finance_Positions.csv'


f = open(filename, "r")


count_comma=0
count_semicolon=0
for i in range(2):
    line = f.readline()
    count_comma=count_comma+line.count(",")
    count_semicolon=count_semicolon+line.count(";")


f.close()

print("Importing Transactions")

sep=","
if(count_semicolon>count_comma):
    sep=";"

#format must be dDD-MM-YYYY
transactions = pd.read_csv(filename,sep=sep,dayfirst=True)

#get the most earliest date
df=pd.DataFrame()
df['Date']=pd.to_datetime(transactions['When'],dayfirst=True)
startdate=min(df['Date']).strftime("%Y-%m-%d")

#get all use tickers distinct
stonks=pd.unique(transactions['Ticker']).tolist()

#get all distinct currencies
stonk_currencies=pd.unique(transactions['Currency']).tolist()

print("Transactions Loaded")

#transactions



Importing Transactions
Transactions Loaded


In [6]:
#donwload all data
data = yf.download(tickers=stonks,start=startdate,interval='1d',group_by='column',prepos=False,threads=True)

#counts entries without values
empty_stocks  =data['Close'].columns[data['Close'].count()==0].to_list()

if len(empty_stocks)>0:
    payload= {}
    payload['stocks'] = empty_stocks
    print("WARNING: The Following Tickers are Invalid: "+" , ".join(empty_stocks)+" , tickers must be set accordingly to  https://finance.yahoo.com/",payload,status='ERROR')

print("Stock Values Loaded")


[*********************100%***********************]  42 of 42 completed
Stock Values Loaded


In [7]:

#analyse which pairs to take and downlaod
allcurrencies=[]

for ticker in stonk_currencies:
    allcurrencies.append(ticker)
    

curs = set(allcurrencies).difference(('USD',))
print(curs)

curpairs=[]

#always converts to dollars
#targetcurrency='USD'

#generate strings
for i in curs:
    curpairs.append(i+"USD=X")


#download required pairs
historicalcurrency = yf.download(tickers=curpairs,start=startdate,interval='1d',group_by='column',prepos=False,threads=True)


#targetpair = yf.download(tickers=targetcurrency+"=X",start=startdate,interval='1d',group_by='column',prepos=False,threads=True)


#Sets to  target currency
historicalcurrency=historicalcurrency['Close']#.mul(targetpair['Close'],axis=0)

{'EUR', 'GBP'}
[*********************100%***********************]  2 of 2 completed


In [15]:
#For every stock,get current format
curtime=datetime.datetime.now().isoformat()

#this may take a while

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=len(stonks)-1,
    description='Fetching:',
    bar_style='info',
    #style={'bar_color': 'maroon'},
    orientation='horizontal'
)

display(progress)

stonks_info={}
dividends={}

inf=0
counter=0;
#serial
for i in stonks:
    try:

        inf=ticker=  yf.Ticker(i).get_info()#defs.FetchTickerInfo(i)
        
        progress.value=counter
       
        
        dividends[i]= yf.Ticker(i).dividends
        
        counter=counter+1;
    except Exception as e:
        print(e)
    stonks_info[i]=inf

    #print(i)
    


IntProgress(value=0, bar_style='info', description='Loading:', max=41)

In [ ]:
stonkdict={}



#initialize final datafraem
superdf =pd.DataFrame()
superdf.empty

i=0


for stonk in stonks:  
    
    print(stonk)
    
    #get transactions from csv
    singlestock_transactions = transactions[transactions['Ticker']==stonk]

    #use index to create df
    index = data.index
    df=pd.DataFrame(index=data.index)

    
    #extract price
    df['Price']= data['Close'][stonk]
    #initialize amount
    df["SharesOwned"]=0
    #initialize total value
    df['TotalValue']=0  
    
    

    df['InvestedValue']=0   

    #set ticker
    df["Ticker"]=stonk
    
    #fetch currency
    df["Currency"]=stonks_info[stonk]['currency']
    if(stonk=='NVDA'):
        print(df.index.max())


    for idx, row in singlestock_transactions.iterrows():

        multiplier = 1
        if(row['What']=='buy'):
            multiplier=1
        elif(row['What']=='sell'):
            multiplier=-1
        else:
            raise Exception("Transaction \"row['What']\" is not recognized")

        #print(row['Amount'])


        #CHECK IF IT GETS APPLIED TO STARTING ARRAY, FIND AWAY TO DO THIS AT THE START
        #CONVERTING INTO AMOUNT MAKES it be adimensional, and doenst matter what currency it was bought in
        try:
            if(math.isnan(float(row['Amount']))):
                row['Amount']=row['Total Cost']/row['Cost per Share']
        except:
            row['Amount']=row['Total Cost']/row['Cost per Share']


        #df[row['When']:][stonk+" Amount"] = df[row['When']:][stonk+" Amount"]=row['Amount']

        #df[row['When']:,stonk+" Amount"] = df[row['When']:,stonk+" Amount"]+row['Amount']

        #print(float(row['Amount']))
        
        wheny = pd.to_datetime(row['When'], dayfirst=True)
        
        if pd.to_datetime("now")< wheny:
            print(row['When'],"<- This date is in the future, and will not be considere")
            continue


        
        #print(float(row['Amount'])*multiplier)
        #set amount to the amount
        #display(row['When'])
        #display(df.loc[row['When']:])
        if(stonk=='NVDA'):
            print(df.index.max())
        
        
        
        
        #change the amount from whtn to forward,  multiplier determinex if its a  buy of a sell
        df.loc[wheny:,'SharesOwned'] = df.loc[wheny:,'SharesOwned'] + (float(row['Amount'])*multiplier)

        #money put in
        df.loc[wheny:,'InvestedValue'] = df.loc[wheny:,'InvestedValue'] + (float(row['Total Cost'])*multiplier)
        

    i=i+1
        
    df.loc[:,'TotalValue']= df["Price"]*df['SharesOwned']
    df.loc[:,'EarnedValue']= df.loc[:,'TotalValue']-df.loc[:,'InvestedValue']
      
        
    #delete rows with no data
    #
    df.interpolate(inplace=True)
    df.dropna(inplace=True,how='any')
    
    
    #display(df)
    #display(df)
    
    
    
    superdf= superdf.append(df)
    #display(df)
    #display(superdf)
    #print("YIKES")

    




In [ ]:
#Check if something is wrong
#for every stock
for stonk in stonks:  
    a = superdf.loc[superdf['Ticker']==stonk]


    if a.loc[a.index.min(),'EarnedValue']>0:
        print('Verify if Stock % has the correct date')




In [ ]:
#reset indexes
superdf=superdf.reset_index()


In [ ]:

#calculate dividends
superdf['Dividends']=0

#for every stock
for stonk in stonks:  
    #print(stonk)
    #stonk="PEP"
    divs=dividends[stonk]

      

    #fetches the start date,crop dividends before that
    divs = divs[startdate:]


    for date, divi in divs.iteritems():

        
        #shares owned when dividends ocurred
        #filter  by stonk and date. fetch values[0]
        sharesOwned = superdf.loc[(superdf['Ticker']==stonk) & (superdf['Date']==date),'SharesOwned'].values[0]

        #debug
        if( stonk=='NVDA'):
            print(date, divi)
            print(sharesOwned)
        #print(date,row)        
        
        #if the dividend date is greater than the date,
        superdf.loc[ (superdf['Date']>=date) & (superdf['Ticker']==stonk),'Dividends']=superdf.loc[ (superdf['Date']>=date) & (superdf['Ticker']==stonk),'Dividends']  + (float(divi)*sharesOwned)
        #display(superdf.loc[superdf["Ticker"]==stonk,'Dividends'].loc[date:])
    

In [ ]:

#needs index to work apparently
superdf = superdf.reset_index(drop=True).set_index(['Date'])

In [ ]:
hm=superdf
hm

#perform left join on Date field, 

superdf = pd.merge(
    hm,
    historicalcurrency,
    how="left",
    left_on='Date',
    right_on='Date',
    left_index=False,
    right_index=False,
    suffixes=("", ""),
    copy=True,
    indicator=False,
    validate=None,
)

#for every different currencu
for (cur,curp) in zip(curs,curpairs):
    #print(cur,curp)
    #interpolate missing data for currency
    superdf[curp]=superdf[curp].interpolate(method='pad')
    
    #convert Everything to USD
    superdf.loc[superdf['Currency']==cur,'TotalValue'] = superdf[superdf['Currency']==cur]['TotalValue']*superdf[superdf['Currency']==cur][curp]
    superdf.loc[superdf['Currency']==cur,'InvestedValue'] = superdf[superdf['Currency']==cur]['InvestedValue']*superdf[superdf['Currency']==cur][curp]
    superdf.loc[superdf['Currency']==cur,'EarnedValue'] = superdf[superdf['Currency']==cur]['EarnedValue']*superdf[superdf['Currency']==cur][curp]
    superdf.loc[superdf['Currency']==cur,'Dividends'] = superdf[superdf['Currency']==cur]['Dividends']*superdf[superdf['Currency']==cur][curp]
   


In [ ]:
superdf

In [ ]:
#superdf[superdf['Ticker']=='XOM']
dfforaltair = superdf.reset_index()
dfforaltair.interpolate(inplace=True)

alt.data_transformers.disable_max_rows()
alt.Chart(dfforaltair).mark_area().encode(
    x='Date',
    y='InvestedValue',
    #color='Ticker:N',
    color=alt.Color('Ticker:N', legend=alt.Legend(columns=1,symbolLimit=300)),
).properties(
    width=1280,
    height=720
)

In [ ]:
superdf.to_csv('PorfolioHistory.csv')

In [ ]:
## Drill down

superdf[superdf['Ticker']=='NVDA']

In [ ]:
superdf

In [ ]:
dividends['NVDA']

In [ ]:
superdf[superdf['Ticker']=='PEP']

In [ ]:
superdf

In [ ]:

#for every stock
for stonk in stonks:  
    print(stonk)
    #stonk="PEP"
    divs=dividends[stonk]

      

    #fetches the start date,crop dividends before that
    divs = divs[startdate:]


    for date, divi in divs.iteritems():

        
        #shares owned when dividends ocurred
        #filter  by stonk and date. fetch values[0]
        sharesOwned = superdf.loc[(superdf['Ticker']==stonk) & (superdf['Date']==date),'SharesOwned'].values[0]

        #debug
        if( stonk=='NVDA'):
            print(date, divi)
            print(sharesOwned)
        #print(date,row)        
        
        #if the dividend date is greater than the date,
        superdf.loc[ (superdf['Date']>=date) & (superdf['Ticker']==stonk),'Dividends']=superdf.loc[ (superdf['Date']>=date) & (superdf['Ticker']==stonk),'Dividends']  + (float(divi)*sharesOwned)
        #display(superdf.loc[superdf["Ticker"]==stonk,'Dividends'].loc[date:])
    

In [ ]:
stok = 'PEP'

min(df.loc[]['Date']).strftime("%Y-%m-%d")